# Opensearch

## Low-level Python client `opensearch-py`

_OpenSearch Python Client Documentation: https://opensearch-project.github.io/opensearch-py/_

In [1]:
import opensearchpy
from opensearchpy import OpenSearch
import json

In [2]:
# Connecting to OpenSearch
# https://opensearch.org/docs/latest/clients/python-low-level/#connecting-to-opensearch

host = 'localhost'
port = 9200
auth = ('admin', 'admin') # For testing only. Don't store credentials in code.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = False, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

In [3]:
# Creating an index: client.indices.create()
# https://opensearch.org/docs/latest/clients/python-low-level/#creating-an-index

index_name = 'python-test-index'
index_body = {
  'settings': {
    'index': {
      'number_of_shards': 2
    }
  }
}

try:
  response = client.indices.create(index_name, body=index_body)
except opensearchpy.exceptions.RequestError as e:
  if 'resource_already_exists_exception' != e.error:
    raise
  else:
    client.indices.delete(index_name)
    response = client.indices.create(index_name, body=index_body)

print(json.dumps(response,indent=2))

{
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "python-test-index"
}


In [4]:
# Indexing a document: client.index()
# https://opensearch.org/docs/latest/clients/python-low-level/#indexing-a-document

document1 = {
  'title': 'Moneyball',
  'director': 'Bennett Miller',
  'year': '2011'
}
document2 = {
  'title': 'Alien Ball',
  'director': 'James Cameron',
  'year': '1978'
}

response = client.index(
  index = 'python-test-index',
  body = document1,
  id = '1',
  refresh = True
)
response = client.index(
  index = 'python-test-index',
  body = document2,
  id = '2',
  refresh = True
)

print(json.dumps(response,indent=2))

{
  "_index": "python-test-index",
  "_id": "2",
  "_version": 1,
  "result": "created",
  "forced_refresh": true,
  "_shards": {
    "total": 2,
    "successful": 1,
    "failed": 0
  },
  "_seq_no": 1,
  "_primary_term": 1
}


In [5]:
# Searching for documents; client.search()
# https://opensearch.org/docs/latest/clients/python-low-level/#searching-for-documents

q = 'CAMERON'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title', 'director']
    }
  }
}

response = client.search(
  body = query,
  index = 'python-test-index'
)

print(json.dumps(response,indent=2))

{
  "took": 40,
  "timed_out": false,
  "_shards": {
    "total": 2,
    "successful": 2,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 1,
      "relation": "eq"
    },
    "max_score": 0.6931471,
    "hits": [
      {
        "_index": "python-test-index",
        "_id": "2",
        "_score": 0.6931471,
        "_source": {
          "title": "Alien Ball",
          "director": "James Cameron",
          "year": "1978"
        }
      }
    ]
  }
}


## Ingest APIs with `opensearch-py`

https://opensearch.org/docs/latest/api-reference/ingest-apis/index/

In [ ]:
import opensearchpy
from opensearchpy import OpenSearch
import json

In [ ]:
# Connecting to OpenSearch
# https://opensearch.org/docs/latest/clients/python-low-level/#connecting-to-opensearch

host = 'localhost'
port = 9200
auth = ('admin', 'admin') # For testing only. Don't store credentials in code.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = False, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

In [ ]:
# Creating a pipline
# https://opensearch.org/docs/latest/api-reference/ingest-apis/create-update-ingest/
# https://kb.objectrocket.com/elasticsearch/how-to-use-the-attachment-plugin-to-index-files-in-elasticsearch-271

pl_id = 'unibe_pipeline'
pl_create_body = {
  "description" : "Field for processing file attachments",
  "processors" : [
    {
      "attachment" : {
        "field": "data",
        "value": "value"
      }
    }
  ]
}


try:
  response = client.indices.create(index_name, body=index_body)
except opensearchpy.exceptions.RequestError as e:
  if 'resource_already_exists_exception' != e.error:
    raise
  else:
    client.indices.delete(index_name)
    response = client.indices.create(index_name, body=index_body)

print(json.dumps(response,indent=2))

{
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "python-test-index"
}
